# 本地模型部署(chatGLM 6b)  

In [ ]:
"""
    源自清华大学 github 网站上的案例（https://github.com/THUDM/ChatGLM-6B）

    调试量化 int4级别

    先保证本地安装调试完CUDA，然后创建conda 环境 进行环境配置 pip install -r requirements.txt -i https://mirror.sjtu.edu.cn/pypi/web/simple

"""
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch")

from transformers import AutoTokenizer, AutoModel
model_path = "D:\CodeLibrary\ChatGLM\chatglm26b"

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
# model = AutoModel.from_pretrained(model_path, trust_remote_code=True, device='cuda')
# 按需修改，目前只支持 4/8 bit 量化
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).quantize(4).half().cuda()
model = model.eval()

response, history = model.chat(tokenizer, "你好", history=[])
print(response + '\n')

# response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
# print(response + '\n')

# 上面这种模型 定义 是不能够放入到 `RetrievalQA` 中的，因为这里的 `model` 是 `AutoModel` 类型，并不是可以 `Runnable` 的

**这样部署的模型直接拿去用是不对的，会出现下面错误：**

```python  
ValidationError: 2 validation errors for LLMChain


llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
```

# 正确的 `model` 定义如下：(这个还有点小问题，后面研究)          

In [1]:
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModel
from typing import Optional, List

class HuggingfaceModel(LLM):
    def __init__(self, model_name: str):
        self.model = AutoModel.from_pretrained(model_name, trust_remote_code=True).eval()
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt")
        output = self.model.generate(input_ids, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)
        return self.tokenizer.decode(output[0], skip_special_tokens=True)

    @property
    def _identifying_params(self) -> dict:
        return {"model_name": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "huggingface"

    def generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> List[str]:
        callback_manager = CallbackManagerForLLMRun.get_instance()
        with callback_manager.as_context():
            callback_manager.on_llm_start({self._llm_type: len(prompts)})
            results = [self._call(prompt, stop) for prompt in prompts]
            callback_manager.on_llm_end({self._llm_type: len(prompts)})
        return results


# 下面这个是没问题的

In [1]:
from langchain.llms.base import LLM
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch")

class ChatGLM_LLM(LLM):
    # 基于本地 InternLM 自定义 LLM 类
    tokenizer : AutoTokenizer = None
    model: AutoModelForCausalLM = None

    def __init__(self, model_path :str):
        # model_path: InternLM 模型路径
        # 从本地初始化模型
        super().__init__()
        print("正在从本地加载模型...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).to(torch.bfloat16).cuda()
        self.model = self.model.eval()
        print("完成本地模型的加载")

    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
        # 重写调用函数
        response, history = self.model.chat(self.tokenizer, prompt , history=[])
        return response
        
    @property
    def _llm_type(self) -> str:
        return "ChatGLM3-6B"

# 本地知识库搭建

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA
import torch

# 加载文件夹中的所有.md类型的文件
loader = DirectoryLoader(r'D:/Notes/NLP review', glob='**/*.md')
# 将数据转成 document 对象，每个文件会作为一个 document
documents = loader.load()

# 初始化加载器
text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
# 切割加载的 document
split_docs = text_splitter.split_documents(documents)

# 指定 Hugging Face 的预训练模型名称
model_name = r"D:\CodeLibrary\ChatGLM\embedtext2vec"  # 示例模型名称，您可以根据需要更改

# 创建 HuggingFaceEmbeddings 对象
embeddings = HuggingFaceEmbeddings(model_name=model_name)

# 构建持久化的 向量数据库    
# vertorstore 持久化地址
persist_directory = r'D:\CodeLibrary\ChatGLM\langchain_QA\chroma_data'

#将数据转化为 embeddings 存入chroma,并设置本地磁盘持久化路径
vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    persist_directory=persist_directory
)

# 持久化 chroma 0.4x版本开始取消了手动 persist,否则会警告，貌似还无法持久化(改为自动持久化了)
# vectordb.persist()

# langchain集成                 
1. 直接导入之前持久化的向量数据库，无需重新构建           
2. 定义直接重写LLM接口的llm          
3. 通过 langchain 提供的 `RetirevalQA` 对象，构建RAG

In [3]:
'''
1. 将 document 通过 HuggingFace 的 embeddings 模型计算 embedding 向量信息并临时存入 Chroma 向量数据库，用于后续匹配查询

2. 后面在 RetrievalQA.from_chain_type 中, 使用 retriever=docsearch.as_retriever()

'''
# docsearch = Chroma.from_documents(split_docs, embeddings)


# 加载 向量数据库           
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

# 初始化自定义 llm 
model_name = r"D:\CodeLibrary\ChatGLM\chatglm26b"

chatglm2 = ChatGLM_LLM(model_name)

正在从本地加载模型...


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

完成本地模型的加载


# 检索问答链 参考代码
```python
store = Chroma.load("chroma_store", embeddings)

template = """基于以下信息来回答用户问题。                          
已知信息：   
{context} 
问题：
{question}"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt":prompt}
qa = RetrievalQA.from_chain_type(llm=chatglm, retriever=store.as_retriever(), chain_type="stuff",
                                chain_type_kwargs=chain_type_kwargs, return_source_documents=True)
```

In [4]:
from langchain import PromptTemplate
template = '''
基于以下信息来回答用户问题。如果你不知道答案，就说你不知道，不要试图编造答案。尽量使答案简单，并最后回答的最后说“谢谢你的提问！”。                      
已知信息： 
{context} 
问题：
{question}
'''

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt":prompt}

# 创建问答对象
# qa = RetrievalQA.from_chain_type(llm=hf_model, chain_type="stuff", vectorstore=docsearch, return_source_documents=True)

qa = RetrievalQA.from_chain_type(
    llm=chatglm2,
    chain_type="stuff",
    chain_type_kwargs=chain_type_kwargs,
    retriever=vectordb.as_retriever(),
    return_source_documents=True
)

# 进行问答 Chain
"""
    类的 __call__ 方法已经被弃用， 用 invoke 方法来代替 __call__ 方法
"""
result = qa.invoke({"query": "什么是RAG？"})
print(result)

{'query': '什么是RAG？', 'result': 'RAG（Reinforcement-Adversarial Generative）是一种基于博弈理论的生成对抗网络，旨在解决生成式任务中的对抗问题。它由两个部分组成：生成器（Generator）和判别器（Discriminator）。\n\n生成器的目标是生成尽可能逼真的样本，以欺骗判别器；判别器的目标是区分真实样本和生成样本，以评估生成器的性能。\n\nRAG 通过对损失函数进行优化，使得生成器能够生成更逼真的样本，同时判别器也能够更好地识别真实样本和生成样本。这种方法在生成式任务中取得了很好的效果，如文本生成、图像生成等。', 'source_documents': [Document(page_content='ELECTRA and RTD任务\n\n参考资料\n\nELECTRA: Pre-training Text Encoders as Discriminators Rather Than Generators\n\n完胜 BERT，谷歌优秀 NLP 预训练模型开源\n\n核心思想 -RTD\n\nELECTRA 使用一种称为替换令牌检测（RTD）的新预训练任务，该任务在从所有输入位置（如：LM）学习的同时，训练双向模型（如：MLM）。\n\n具体而言，ELECTRA 的目标是学习区分输入的词。它不使用掩码，而是从一个建议分布中采样词来替换输入，这解决了掩码带来的预训练和 fine-tune 不一致的问题。\n\n然后模型再训练一个判别器，来预测每个词是原始词还是替换词。而判别器的一个优点则是： ==模型从输入的所有词中学习==，而不是像 MLM 那样，仅使用掩盖的词，因此计算更加有效。\n\n正如很多开发者联想到的对抗学习方法，ELECTRA 确实受到到生成对抗网络的启发（GAN）。但不同的是，模型采用的是最大似然而非对抗学习。\n\n例如下图中，单词「cooked」可以替换为「ate」。尽管这有些道理，但它并不适合整个上下文。预训练任务需要模型（即鉴别器）来确定原始输入中的哪些标记已被替换或保持相同。\n\n正是由于该模型的二进制分类任务适用于每个输入单词，而非仅有少量的掩码单词（在 BERT 样式的模型中为 15％），因此，RTD 方法的效率比 MLM 高。这也解释了为什么 ELEC

In [5]:
print(result.keys())

dict_keys(['query', 'result', 'source_documents'])


In [6]:
print(result['query'])

什么是RAG？


In [7]:
print(result['result'])

RAG（Reinforcement-Adversarial Generative）是一种基于博弈理论的生成对抗网络，旨在解决生成式任务中的对抗问题。它由两个部分组成：生成器（Generator）和判别器（Discriminator）。

生成器的目标是生成尽可能逼真的样本，以欺骗判别器；判别器的目标是区分真实样本和生成样本，以评估生成器的性能。

RAG 通过对损失函数进行优化，使得生成器能够生成更逼真的样本，同时判别器也能够更好地识别真实样本和生成样本。这种方法在生成式任务中取得了很好的效果，如文本生成、图像生成等。


In [8]:
print(result["source_documents"])

[Document(page_content='ELECTRA and RTD任务\n\n参考资料\n\nELECTRA: Pre-training Text Encoders as Discriminators Rather Than Generators\n\n完胜 BERT，谷歌优秀 NLP 预训练模型开源\n\n核心思想 -RTD\n\nELECTRA 使用一种称为替换令牌检测（RTD）的新预训练任务，该任务在从所有输入位置（如：LM）学习的同时，训练双向模型（如：MLM）。\n\n具体而言，ELECTRA 的目标是学习区分输入的词。它不使用掩码，而是从一个建议分布中采样词来替换输入，这解决了掩码带来的预训练和 fine-tune 不一致的问题。\n\n然后模型再训练一个判别器，来预测每个词是原始词还是替换词。而判别器的一个优点则是： ==模型从输入的所有词中学习==，而不是像 MLM 那样，仅使用掩盖的词，因此计算更加有效。\n\n正如很多开发者联想到的对抗学习方法，ELECTRA 确实受到到生成对抗网络的启发（GAN）。但不同的是，模型采用的是最大似然而非对抗学习。\n\n例如下图中，单词「cooked」可以替换为「ate」。尽管这有些道理，但它并不适合整个上下文。预训练任务需要模型（即鉴别器）来确定原始输入中的哪些标记已被替换或保持相同。\n\n正是由于该模型的二进制分类任务适用于每个输入单词，而非仅有少量的掩码单词（在 BERT 样式的模型中为 15％），因此，RTD 方法的效率比 MLM 高。这也解释了为什么 ELECTRA 只需更少的示例，就可以达到与其它语言模型相同性能的原因。\n\n具体架构\n\n替换令牌来自生成器的神经网络。生成器的目标是训练掩码语言模型，即给定输入序列后，按照一定的比例（通常 15%）将输入中的词替换成掩码；然后通过网络得到向量表示；之后再采用 softmax 层，来预测输入序列中掩盖位置的词。\n\n尽管生成器的结构类似于 GAN，但由于难以将该方法应用于文本任务，因此得到的训练目标函数为掩盖词的最大似然。\n\n之后，生成器和判别器共享相同的输入词嵌入。判别器的目标是判断输入序列每个位置的词是否被生成器替换，如果与原始输入序列对应位置的词不相同，就判别为已替换。\n\n补充\n\n在Electra模型中，RTD（Re

In [9]:
import inspect

print(inspect.getsource(RetrievalQA.from_chain_type))

    @classmethod
    def from_chain_type(
        cls,
        llm: BaseLanguageModel,
        chain_type: str = "stuff",
        chain_type_kwargs: Optional[dict] = None,
        **kwargs: Any,
    ) -> BaseRetrievalQA:
        """Load chain from chain type."""
        _chain_type_kwargs = chain_type_kwargs or {}
        combine_documents_chain = load_qa_chain(
            llm, chain_type=chain_type, **_chain_type_kwargs
        )
        return cls(combine_documents_chain=combine_documents_chain, **kwargs)



# 看了下知识库中的内容再进行一次查询发现是有效的（之前效果不好可能是因为知识库的原因，以及部署chatglm6b 我选择的精度比较低）,还可能是因为我使用 CharacterTextSplitter 的时候 overlap设置的为0

In [12]:
a = qa.invoke({'query':'什么是RAG 融合'})
print(a['result'])

RAG 融合是一种将检索和生成过程进行有效整合的方法，旨在保留检索和生成各自的优势，并更加有效地利用两者之间的关系，从而提高整体系统的性能和效果。它可以通过以下几种方式实现：

1. 交互式融合：在检索和生成之间建立一种交互式的机制，让它们可以相互影响和调整。例如，在生成阶段可以考虑将生成的内容作为反馈信息反过来影响检索过程，以提高检索的准确性。
2. 信息传递：在检索到相关文档后，将一些关键信息传递给生成模型，以帮助生成更加相关和准确的答案。这些信息可以是关键词、上下文信息等。
3. 多阶段生成：将生成过程分为多个阶段，每个阶段都与检索结果有关，逐步细化生成的内容。例如，先生成一个粗略的答案，然后根据检索到的文档进一步完善和细化答案。
4. 端到端训练：将检索和生成过程作为一个整体进行端到端的训练，以更好地优化两者之间的关系，使得模型能够更好地学习到检索和生成之间的互相影响。

通过 RAG 融合，可以更好地整合检索和生成两个阶段，充分发挥它们各自的优势，从而提高整体系统的性能和效果。


In [11]:
print(a['source_documents'])

[Document(page_content='RAG Fusion\n\nRAG Fusion 参考，还讲了 RRF算法\n\n概念\n\nRAG 融合（RAG Fusion）是指在 RAG 模型中将检索和生成两个阶段进行有效整合的过程。在传统的 RAG 模型中，首先进行文档检索以获取相关文档，然后使用生成模型根据这些文档生成答案。但在某些情况下，这种简单的串行方法可能无法充分利用检索和生成之间的互补性。\n\nRAG 融合的目的是在保留检索和生成各自优势的同时，更加有效地利用两者之间的关系，从而提高整体的性能。这可以通过以下几种方式来实现：\n\n交互式融合：在检索和生成之间建立一种交互式的机制，让它们可以相互影响和调整。例如，在生成阶段可以考虑将生成的内容作为反馈信息反过来影响检索过程，以提高检索的准确性。\n\n信息传递：在检索到相关文档后，将一些关键信息传递给生成模型，以帮助生成更加相关和准确的答案。这些信息可以是关键词、上下文信息等。\n\n多阶段生成：将生成过程分为多个阶段，每个阶段都与检索结果有关，逐步细化生成的内容。例如，先生成一个粗略的答案，然后根据检索到的文档进一步完善和细化答案。\n\n端到端训练：将检索和生成过程作为一个整体进行端到端的训练，以更好地优化两者之间的关系，使得模型能够更好地学习到检索和生成之间的互相影响。\n\n通过 RAG 融合，可以更好地整合检索和生成两个阶段，充分发挥它们各自的优势，从而提高整体系统的性能和效果。\n\n流程图部分\n\n之前的 多重查询之后， 可能会存在很多相似文档，给大模型之前肯定还是需要做些处理，故上面流程图中，文档与大模型之间的部分，就是 Fusion。这里涉及到一个 算法(Reciprocal ran fusion)，这是一个对文档进行打分的一个算法\n\nreciprocal rank fusion\n\n案例流程图\n\n计算公式\n$$RRFscore(d\\in D)=\\sum_{r\\in R}\\frac1{k+r(d)} ,$$\n\n来源（https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf）', metadata={'source': 'D:\\Notes\\NLP review\\RAG related\\RAG

In [13]:
print(a['query'])

什么是RAG 融合


In [13]:
from langchain.llms.base import LLM, BaseLLM
from langchain.vectorstores import Chroma
import inspect

print(inspect.getsource(Chroma.from_documents))

    @classmethod
    def from_documents(
        cls: Type[Chroma],
        documents: List[Document],
        embedding: Optional[Embeddings] = None,
        ids: Optional[List[str]] = None,
        collection_name: str = _LANGCHAIN_DEFAULT_COLLECTION_NAME,
        persist_directory: Optional[str] = None,
        client_settings: Optional[chromadb.config.Settings] = None,
        client: Optional[chromadb.Client] = None,  # Add this line
        collection_metadata: Optional[Dict] = None,
        **kwargs: Any,
    ) -> Chroma:
        """Create a Chroma vectorstore from a list of documents.

        If a persist_directory is specified, the collection will be persisted there.
        Otherwise, the data will be ephemeral in-memory.

        Args:
            collection_name (str): Name of the collection to create.
            persist_directory (Optional[str]): Directory to persist the collection.
            ids (Optional[List[str]]): List of document IDs. Defaults to None.
          

# 仅用大模型回答看看答案

In [10]:
llm_only_result = chatglm2("什么是RAG")  

# 可以明显看到，没有本地知识库的帮助，答案是非常糟糕的（虽然有本地知识库也有错误。）

In [11]:
print(llm_only_result)

RAG是指残骸（R骸），通常是指在战争、自然灾害或其他暴力事件中失去生命的动物或人类遗体。这些遗体可能被遗弃在公共场所，例如街道、广场或公园等。 RAG对于战时医疗工作者来说，是一个重要的词汇，意味着需要立即处理和妥善处理这些遗体，以防止疾病传播和环境污染。


In [15]:
llm_only_result2 = chatglm2("什么是RAG融合？")
print(llm_only_result2)

RAG融合（RAG-Fusion）是一种将来自不同RAG（Read-Ahead Generation）技术的数据进行融合的技术，旨在提高数据处理的效率和准确性。通过将来自多个RAG的数据进行融合，可以避免在处理过程中产生重复的信息，从而提高数据的一致性和完整性。

RAG融合技术可以在多种应用场景中使用，如在分布式文件系统、大数据处理系统、数据库系统中等。常见的RAG包括：

1. 基于哈希的RAG（如Hadoop Distributed File System中的RAG）
2. 基于内容的RAG（如HBase中的RAG）
3. 基于时间的RAG（如TimeSeriesDB中的RAG）

RAG融合方法可以有多种，如简单的拼接、基于统计的融合、基于机器学习的融合等。在实际应用中，需要根据具体场景和需求选择合适的RAG融合方法。


# 部署 Web Demo   
根据之前的到的结果：
```python
result = qa.invoke({"query": "什么是RAG？"})
print(result)  
```
可以将这些内容进行封装，通过gradio 进行web部署，主要流程如下:
1. 将之前的langchain问答链相关部分进行封装变成一个对象  
2. 启动gradio，通过该对象进行知识问答      

## 先进行封装

In [ ]:

from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
import os
from LLM import ChatGLM_LLM
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

def load_chain():
    # 加载问答链
    # 定义 Embeddings
    embeddings = HuggingFaceEmbeddings(model_name=r"D:\CodeLibrary\ChatGLM\embedtext2vec")

    # 向量数据库持久化路径
    persist_directory = r'D:\CodeLibrary\ChatGLM\langchain_QA\chroma_data'

    # 加载数据库
    vectordb = Chroma(
        persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
        embedding_function=embeddings
    )

    # 加载自定义 LLM
    chatglm = ChatGLM_LLM(model_path = r"D:\CodeLibrary\ChatGLM\chatglm26b")

    # 定义一个 Prompt Template
    template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
    案。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。 
    {context} 
    问题: {question} 
    有用的回答:""" 

    QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],template=template)


    # 运行 chain 
    qa_chain = RetrievalQA.from_chain_type(chatlm,retriever=vectordb.as_retriever(),return_source_documents=True,chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

    
    return qa_chain


# 该类负责加载并存储检索问答链，并响应 Web 界面里调用检索问答链进行回答的动作
class Model_center():
    """
    存储检索问答链的对象 
    """
    def __init__(self):
        # 构造函数，加载检索问答链
        self.chain = load_chain()

    def qa_chain_self_answer(self, question: str, chat_history: list = []):
        """
        调用问答链进行回答
        """
        if question == None or len(question) < 1:
            return "", chat_history
        try:
            chat_history.append(
                (question, self.chain({"query": question})["result"]))
            # 将问答结果直接附加到问答历史中，Gradio 会将其展示出来
            return "", chat_history
        except Exception as e:
            return e, chat_history


## gradio 部分  

In [ ]:
import gradio as gr

# 实例化核心功能对象
model_center = Model_center()
# 创建一个 Web 界面
block = gr.Blocks()
with block as demo:
    with gr.Row(equal_height=True):   
        with gr.Column(scale=15):
            # 展示的页面标题
            gr.Markdown("""<h1><center>基于ChatGLM2的本地知识问答</center></h1>
                <center>base ChatGLM2-6B + chroma</center>              
                """)  

    with gr.Row():
        with gr.Column(scale=4):
            # 创建一个聊天机器人对象
            chatbot = gr.Chatbot(height=450, show_copy_button=True)
            # 创建一个文本框组件，用于输入 prompt。
            msg = gr.Textbox(label="Prompt/问题")

            with gr.Row():
                # 创建提交按钮。
                db_wo_his_btn = gr.Button("Chat")
            with gr.Row():
                # 创建一个清除按钮，用于清除聊天机器人组件的内容。
                clear = gr.ClearButton(
                    components=[chatbot], value="Clear console")
                
        # 设置按钮的点击事件。当点击时，调用上面定义的 qa_chain_self_answer 函数，并传入用户的消息和聊天历史记录，然后更新文本框和聊天机器人组件。
        db_wo_his_btn.click(model_center.qa_chain_self_answer, inputs=[
                            msg, chatbot], outputs=[msg, chatbot])
        
    gr.Markdown("""提醒：<br>
    1. 初始化数据库时间可能较长，请耐心等待。
    2. 使用中如果出现异常，将会在文本输入框进行展示，请不要惊慌。 <br>
    """)
gr.close_all()
# 直接启动
demo.launch()